# Load libraries

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import h5py
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import Conv3D, Conv2D, ConvLSTM2D, BatchNormalization, LeakyReLU
from PIL import Image
import glob
from IPython import display
import matplotlib.animation as animation
matplotlib.use("Agg")
import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
from PIL import Image
from IPython.display import HTML
from PIL import Image, ImageDraw
import sklearn.model_selection as sk

# Load dataset

In [2]:
!dir

 Volume in drive E is New Volume
 Volume Serial Number is A861-2BE5

 Directory of e:\2_TY\DL\DL-CP\LSTM-radar-precipitation-forecast-main

23-04-2024  16:43    <DIR>          .
23-04-2024  16:17    <DIR>          ..
23-04-2024  13:16                49 .gitignore
23-04-2024  17:46    <DIR>          .venv
23-04-2024  18:35    <DIR>          data
08-04-2024  13:48       996,176,770 data.zip
23-04-2024  17:10             3,573 download_raw_data.py
08-04-2024  16:30           341,951 ground_truth.gif
23-04-2024  13:16               108 implinks.txt
23-04-2024  16:44             5,952 lstm_model.py
08-04-2024  16:30           135,615 model.png
08-04-2024  19:13    <DIR>          model_saved
08-04-2024  16:30           384,136 predicted.gif
22-04-2024  21:56             1,770 raw_data_to_imges.py
08-04-2024  16:30             1,198 README.md
23-04-2024  16:50             3,903 Unet_mode.py
08-04-2024  16:30             3,137 validation.py
23-04-2024  18:41            59,732 weather_LSTM.ipyn

In [3]:
import numpy as np
import os
import h5py
from PIL import Image
import sklearn.model_selection as sk

def create_dataset_from_raw(directory_path, resize_to):
    resize_width = resize_to[0]
    resize_height = resize_to[1]
    batch_names = [os.path.join(directory_path, name) for name in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, name))]
    dataset = np.zeros(shape=(len(batch_names), 36, resize_height, resize_width))  # (samples, filters, rows = height, cols = width)

    for batch_idx, batch in enumerate(batch_names):
        files = [x for x in os.listdir(batch) if x != '.DS_Store']
        files.sort()
        crn_batch = np.zeros(shape=(36, resize_height, resize_width)) 
        for idx, raster in enumerate(files):
            fn = os.path.join(batch, raster)
            img = h5py.File(fn)
            original_image = np.array(img["image1"]["image_data"]).astype(float)
            img = Image.fromarray(original_image)
            img = img.resize(size=(resize_width, resize_height))
            original_image = np.array(img)
            original_image = original_image / 255.0
            crn_batch[idx] = original_image
        dataset[batch_idx] = crn_batch
        print("Importing batch: " + str(batch_idx))
    return dataset

def create_shifted_frames(data):
    x = data[:, :18, :, :]
    y = data[:, 18:36, :, :]
    return x, y

# Specify the directory path and resize dimensions
directory_path = './data/data/raw_training/'
resize_to = (315, 344)

# Create the dataset from raw images
dataset = create_dataset_from_raw(directory_path, resize_to)

# Expand dimensions to include channel dimension
dataset = np.expand_dims(dataset, axis=-1)

# Split dataset into input frames (X) and output frames (y)
X, y = create_shifted_frames(dataset)

# Split X and y into training and validation sets
X_train, X_val, y_train, y_val = sk.train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)


Importing batch: 0
Importing batch: 1
Importing batch: 2
Importing batch: 3
Importing batch: 4
Importing batch: 5
Importing batch: 6
Importing batch: 7
Importing batch: 8
Importing batch: 9
Importing batch: 10
Importing batch: 11
Importing batch: 12
Importing batch: 13
Importing batch: 14
Importing batch: 15
Importing batch: 16
Importing batch: 17
Importing batch: 18
Importing batch: 19
Importing batch: 20
Importing batch: 21
Importing batch: 22
Importing batch: 23
Importing batch: 24
Importing batch: 25
Importing batch: 26
Importing batch: 27
Importing batch: 28
Importing batch: 29
Importing batch: 30
Importing batch: 31
Importing batch: 32
Importing batch: 33
Importing batch: 34
Importing batch: 35
Importing batch: 36
Importing batch: 37
Importing batch: 38
Importing batch: 39
Importing batch: 40
Importing batch: 41
Importing batch: 42
Importing batch: 43
Importing batch: 44
Importing batch: 45
Importing batch: 46
Importing batch: 47
Importing batch: 48
Importing batch: 49
Importing 

MemoryError: Unable to allocate 4.65 GiB for an array with shape (320, 18, 344, 315, 1) and data type float64

In [ ]:
# import os
# import numpy as np
# import sklearn.model_selection as sk

# def load_images_from_folder(folder_path, resize_to):
#     images = []
#     for filename in os.listdir(folder_path):
#         img_path = os.path.join(folder_path, filename)
#         if os.path.isfile(img_path):
#             # Load and resize the image
#             img = load_and_resize_image(img_path, resize_to)
#             images.append(img)
#     return np.array(images)

# def load_and_resize_image(img_path, resize_to):
#     # Load the image using PIL and resize it to the specified dimensions
#     img = Image.open(img_path)
#     img = img.resize(resize_to)
#     # Convert the image to numpy array and normalize pixel values
#     img = np.array(img) / 255.0
#     return img

# # Directory paths for training and testing images
# train_folder = "./data/images/raw_training/"
# test_folder = "./data/images1/raw_validation/"

# # Resize dimensions for the images
# resize_to = (315, 344)

# # Load training and testing images
# X_train = load_images_from_folder(train_folder, resize_to)
# X_test = load_images_from_folder(test_folder, resize_to)

# # Assuming y_train and y_test are the corresponding labels or output frames
# # You need to adjust this part based on your data structure

# # Split the data into training and testing sets
# y_train = None  # Replace None with your actual labels for training set
# y_test = None   # Replace None with your actual labels for testing set

# # Print the shapes of the training and testing sets
# print("X_train shape:", X_train.shape)
# print("y_train shape:", y_train.shape if y_train is not None else "Not provided")
# print("X_test shape:", X_test.shape)
# print("y_test shape:", y_test.shape if y_test is not None else "Not provided")


# Model

In [ ]:
from keras.layers import Dense
def create_model():
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(7, 7),
                    input_shape=(18,344,315,1),
                    padding='same',activation=LeakyReLU(alpha=0.01), 
                    return_sequences=True))
    model.add(BatchNormalization())
    model.add(ConvLSTM2D(filters=64, kernel_size=(5, 5),
                    padding='same',activation=LeakyReLU(alpha=0.01), 
                    return_sequences=True))
    model.add(BatchNormalization())
    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                    padding='same',activation=LeakyReLU(alpha=0.01), 
                    return_sequences=True))
    model.add(BatchNormalization())
    model.add(ConvLSTM2D(filters=64, kernel_size=(1, 1),
                    padding='same',activation=LeakyReLU(alpha=0.01), 
                    return_sequences=True))
    model.add(BatchNormalization())
    model.add(Conv3D(filters=1, kernel_size=(3,3,3),
                activation='sigmoid',
                padding='same', data_format='channels_last'))
    return model

model = create_model()

model.compile(loss='binary_crossentropy', optimizer='adadelta')
print(model.summary())


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d_4 (ConvLSTM2D)      │ (None, 18, 344, 315,   │       815,616 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 18, 344, 315,   │           256 │
│ (BatchNormalization)            │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_5 (ConvLSTM2D)      │ (None, 18, 344, 315,   │       819,456 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 18, 344, 315,   │           256 │
│ (BatchNormalization)            │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_6 (ConvLSTM2D)      │ (None, 18, 344, 315,   │       295,168 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 18, 344, 315,   │           256 │
│ (BatchNormalization)            │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_7 (ConvLSTM2D)      │ (None, 18, 344, 315,   │        33,024 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 18, 344, 315,   │           256 │
│ (BatchNormalization)            │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 18, 344, 315,   │         1,729 │
│                                 │ 1)                     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,966,017 (7.50 MB)

 Trainable params: 1,965,505 (7.50 MB)

 Non-trainable params: 512 (2.00 KB)

None


In [ ]:
# Define modifiable training hyperparameters.
epochs = 25
batch_size = 1

#Fit the model to the training data.
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    verbose=1,
)
model.save('./model_saved')

# Predict values

In [ ]:
# reconstructed_model = keras.models.load_model("./drive/MyDrive/model_saved")
# pick a random index from validation dataset
random_index = np.random.choice(range(len(X_val)), size=1)
test_serie_X = X_val[random_index[0]]
test_serie_Y = y_val[random_index[0]]

first_frames = test_serie_X
original_frames = test_serie_Y
# predict the next 18 fames
new_prediction = model.predict(np.expand_dims(first_frames, axis=0))
new_prediction = np.squeeze(new_prediction, axis=0)

fig, axes = plt.subplots(2, 18, figsize=(20, 4))

# Plot the original frames.
for idx, ax in enumerate(axes[0]):
    ax.imshow(np.squeeze(original_frames[idx]), cmap="viridis")
    ax.set_title(f"Frame {idx + 18}")
    ax.axis("off")

# Plot the predicted frames.
for idx, ax in enumerate(axes[1]):
    ax.imshow((new_prediction[idx]).reshape((344,315)), cmap="viridis")
    ax.set_title(f"Frame {idx + 18}")
    ax.axis("off")

# Display the figure.
plt.show()


# Create animated GIFs for original frames

In [ ]:
fig, ax = plt.subplots()
original_images = []
for f in original_frames:
    ax.set_title(f"Ground Truth")
    ax.axis("off")
    crn_f = ax.imshow(np.squeeze(f),cmap='viridis', animated=False)
    original_images.append([crn_f])
animation_originals = animation.ArtistAnimation(fig, original_images, 
                                                interval=100, blit=False, 
                                                repeat_delay=1000)
animation_originals.save('./drive/MyDrive/ground_truth.gif', 
                         writer=animation.PillowWriter(), dpi=100)
HTML(animation_originals.to_html5_video())

# Create animated GIF for predicted frames

In [ ]:
fig, ax = plt.subplots()
predicted_images = []
for f in new_prediction:
    ax.set_title(f"Ground Truth")
    ax.axis("off")
    crn_f = ax.imshow(np.squeeze(f),cmap='viridis', animated=False)
    ax.set_title(f"Predicted frames")
    predicted_images.append([crn_f])
print(len(predicted_images))
animation_predicted = animation.ArtistAnimation(fig, predicted_images, 
                                                interval=100, blit=False, 
                                                repeat_delay=1000)
animation_predicted.save('./drive/MyDrive/predicted.gif', 
                         writer=animation.PillowWriter(), dpi=100)
HTML(animation_predicted.to_html5_video())